# Detect and recognize images

This notebook shows the process to use the trained model to detect fruits and flowers in herabrium sample pictures. For simplicity you can run the detect_herbaria script using a command line interface:

TODO: ADD example

In [ ]:
# Import modules and packages
import os
import sys
import json
import numpy as np
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
plt.style.context("fivethirtyeight")
from skimage.io import imsave, imread


In [ ]:
# connect to gdrive # TO BE REMOVED
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/gdrive', force_remount=False)

Mounted at /gdrive


### Directory paths

Fill in & test the directory needed for inference and data analysis:

- ROOT_DIR is the folder where the Mask_RCNN model is saved together with the settings

- PROJECT_DIR (full path) is the folder where the images are located and where the output of the model will be saved

- INPUT_DIR_NAME is a subdirectory of the PROJECT_DIR where the images that you want to process are

- INPUT_ANNOTATIONS_FILE is the (OPTIONAL) name of the JSON files with the annotations in COCO style format

- OUTPUT_DIR_NAME is a subdirectory of the PROJECT_DIR where the output information and (OPTIONALLY) the masks will be saved




In [ ]:
# directory of input and output images 
PROJECT_DIR = "/gdrive/My Drive/brassicacea_automatic_classification/coding/ETH_DATASET"
assert os.path.exists(PROJECT_DIR), 'PROJECT_DIR does not exist. Did you forget to read the instructions above? ;)'
%cd {PROJECT_DIR}

# MASK R CNN dir
ROOT_DIR =  '/gdrive/My Drive/brassicacea_automatic_classification/coding/herbaria--plant-labeling/src/Mask_RCNN'
assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'
 

INPUT_DIR_NAME= "preproc/test"
INPUT_ANNOTATIONS_FILE = "preproc/test/test.json"
OUTPUT_DIR_NAME = "OUTPUT_images_MASK_R_CNN"

/gdrive/My Drive/brassicacea_automatic_classification/coding/ETH_DATASET


In [ ]:
# import custom modules from herbaria.py
#PROBALY after is sys.path.append("../src")
# import brassicas as bs
import sys
path=os.path.join( os.path.abspath(os.path.join(ROOT_DIR, os.pardir)))
sys.path.append(path)
import herbaria


AssertionError: ignored

In [ ]:

import helper_functions as hp

# import mask-r-cnn modules and libraries. 
# These are the files that make the model work. If not available they can be downloaded 
# from : https://github.com/akTwelve/Mask_RCNN. The notebvbok will attempt to do it automatically

sys.path.append(ROOT_DIR) 
try:
  from mrcnn.config import Config
  import mrcnn.utils as utils
  from mrcnn import visualize
  import mrcnn.model as modellib

except ModuleNotFoundError:
  print("modules for MaskRCNN not found, downloading mask-RCNN from source")
  !git clone https://github.com/akTwelve/Mask_RCNN

  sys.path.append(os.path.join(ROOT_DIR, Mask_RCNN))
  from mrcnn.config import Config
  import mrcnn.utils as utils
  from mrcnn import visualize
  import mrcnn.model as modellib



Below are the configs to create the model and the dataset. They are based on the main class of the Mask_RCNN utils.Config() and are present in the herbaria.py module. Here we show the full config. Then we actually import it from herbaria.py


In [ ]:
class HerbariaConfig(Config):
    """Configuration for training on the cigarette butts dataset.
    Derives from the base Config class and overrides values specific
    to the cigarette butts dataset.
    """
    # Give the configuration a recognizable name
    NAME = "M_image_augm"

    # Train on 1 GPU and 1 image per GPU. Batch size is 1 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2   # background + 2 [ 'flower', fruit]

    # All of our training images are 1024x1024
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024

    # You can experiment with this number to see if it improves training
    STEPS_PER_EPOCH = 4

    # This is how often validation is run. If you are using too much hard drive space
    # on saved models (in the MODEL_DIR), try making this value larger.
    VALIDATION_STEPS = 1
    
    # Matterport originally used resnet101, but I downsized to fit it on my graphics card
    BACKBONE = 'resnet50'

    # To be honest, I haven't taken the time to figure out what these do
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 50 
    POST_NMS_ROIS_INFERENCE = 500 
    POST_NMS_ROIS_TRAINING = 1000 
    LEARNING_RATE=0.01
    
config = HerbariaConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.01
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE          

The following are additional settings for inference, based on the **HerbariaConfig** class

In [ ]:

class InferenceConfig(HerbariaConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    IMAGE_MIN_DIM = 1024
    IMAGE_MAX_DIM = 1024
    DETECTION_MIN_CONFIDENCE = 0.5
    
inference_config = InferenceConfig()

# DATASET creation

The HerbariaDataset class holds the procedure for creating a dataset to feed into the model (OPTIONAL: suggested only for model evaluation and training). It can be imported from herbaria.py as well with the call
`from herbaria import HerbariaDataset`


In [ ]:
class HerbariaDataset(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """
    def load_data(self, annotation_json, images_dir):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """
        # Load json from file
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()
        
        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']
            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(class_name))
                return
            
            self.add_class(source_name, class_id, class_name)
        
        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)
        
        # Get all images and add them to the dataset
        seen_images = {}
        for image in coco_json['images']:
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))
                
                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]
                
                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )
                
    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []
        
        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        
        return mask, class_ids

### Folders and model settings

In [ ]:
# Set the ROOT_DIR variable to the root directory of the Mask_RCNN model

assert os.path.exists(ROOT_DIR), 'ROOT_DIR does not exist. Did you forget to read the instructions above? ;)'

# Set the data sources for images and OPTIONAL annotations /ground truth (for testing set)
fpath_test_annotations = os.path.join(PROJECT_DIR,INPUT_ANNOTATIONS_FILE)
fldr_path_test_images = os.path.join(PROJECT_DIR, INPUT_DIR_NAME)

# Folder for the model weights
MODEL_DIR = os.path.join("logs/Mask_RCNN")


### Load data


In [ ]:
# ROOT_DIR =  'Mask_RCNN'
# # Import mrcnn libraries
# sys.path.append(ROOT_DIR) 
# from mrcnn.config import Config
# import mrcnn.utils as utils
# from mrcnn import visualize
# import mrcnn.model as modellib

In [ ]:
## THIS COULD BE REMOVED PROBABLY
# Load dataset
dataset = HerbariaDataset()

dataset.load_data(fpath_test_annotations, fldr_path_test_images)
dataset.prepare()

### Create model

Here we first create the model using the HerbariaConfig settings, than we load the model using the trained_weights argument.
This is equivalent to the `herbaria.load_trained_model` function

In [ ]:
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

In [ ]:
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
# model_path = model.find_last()
model_path = "../logs/Mask_RCNN/m_image_augm20201116T0937/mask_rcnn_m_image_augm_0009.h5"

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# load_trained_model function
def load_trained_model(model_path, mode="inference", config=None):
  """function to load a model with pre_trained weights.
  Params:
  - model_path : STRING path saved weights
  - mode : STRING "inference" or "train" DEFAULT "inference"
  - config :  object of class Config, defaults to InferenceConfig for inference and HerbariaConfig for training
  Returns a pre-trained model ready for training or inference"""
  
  assert os.path.exists(model_path), "Provide path to trained weights"

  if config == None:
    if mode == 'train':
      config = HerbariaConfig()
    elif mode == 'inference':
      config = InferenceConfig()
    else :     
        assert " 'mode' should be 'train' or 'inference' "

  print("Loading weights from ", model_path)
  model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)
  # load weights
  model.load_weights(model_path, by_name=True)
  return model
  
model_path = "../logs/Mask_RCNN/m_image_augm20201116T0937/mask_rcnn_m_image_augm_0009.h5"
model=load_trained_model(model_path)

Loading weights from  ../logs/Mask_RCNN/m_image_augm20201116T0937/mask_rcnn_m_image_augm_0009.h5
Instructions for updating:
Use fn_output_signature instead
Re-starting from epoch 9


## Load images and run inference using the model


In [ ]:
# todo ADD THRESHOLD FOR ACCEPTABLE PREDICTION HERE

### Settings for inference

The following are the settings to run the inference

In [ ]:
# Set to true to display all images !! Can be slow with many images
show_predictions = True

# Set to true if you want to save detction outputs
save_predictions = False
#Set to true if you want to save the mask as image. This can be memory intensive
save_masks = False
# Set to true if you want to filter the prediction ba scores. Suggested threshold 0.5
filter_scores= True 
filter_score_thresh=0.5
assert filter_score_thresh >0, "Set filter score above 0"



  

In [ ]:
import skimage
input_images = os.path.join(PROJECT_DIR,INPUT_DIR_NAME)
output_folder= os.path.join(PROJECT_DIR,OUTPUT_DIR_NAME)

def detect_flowers_fruits(input_images_dir, output_folder, model, **kwargs):
  """ Runs predictions on images and save output in folder. 
  Optionally can:
       Filter the outputs based on the score (confidence of the predictions),
       Save masks as png,
       Display images and outputs  

  Returns the filename of the images processed correctly

  Params:
  -input_images_dir: abs path to folder of input images
  -output_folder: abs path to output destination folder
  -model : model to use to run predictions
  OPTIONAL params:
  -save_predictions: BOOL default True  
  -save masks: BOOL default False
  -filter_scores: BOOL default True
  -filter_score_thresh: Decimal between 0 and 1 default 0.5"""


  save_predictions=kwargs.get("save_predictions", True)
  save_masks=kwargs.get("save_masks", False)
  filter_scores=kwargs.get("filter_scores", True, )
  filter_score_thresh=kwargs.get("filter_score_thresh", 0.5, )
  show_predictions=kwargs.get("show_predictions", False )

  print(f" running inference with the following options:")
  print(f" image source : ", input_images_dir)
  print(f" destination folder: {output_folder}")
  for k,v in kwargs.items():
    print(k,v)

  # checks
  assert os.listdir(input_images_dir), "input folder is empty. Please check path or extension of files"
  if not os.path.exists(output_folder):
    'dest folder does not exist. It will be created ;)'
    os.mkdir(output_folder) 

  processed_list=[]
  for filename in os.listdir(input_images)[:2]:
      if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
          image_path=os.path.join(input_images, filename)
      else: 
        continue

      img = skimage.io.imread(image_path)
      img_arr = np.array(img)
      results = model.detect([img_arr], verbose=0) # this is equivalent to .predict() and actually does the inference
      # filter by scores
      
      r = results[0] # this contains all the predictions including the masks for one image
      
      if filter_scores:
        print(f"removing predictions with confidence below {filter_score_thresh}")
        
        sc_=r['scores']
        lim_=len(sc_[ r['scores'] > filter_score_thresh ])-1
        r['masks']=r['masks'][:,:,:lim_]
        r['rois']=r['rois'][:lim_]
        r['scores']= r['scores'][:lim_]
        r['class_ids'] = r['class_ids'][:lim_]
    
      # display images
      if show_predictions:
        visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                  dataset.class_names,figsize=(100,100))
      # save images
      if save_predictions:
        print(f"\n saving predictions for image {filename} to {output_folder}")
        # create dictionary ( FOR NOW MASKS are EXCLUDED)
        json_pred={}
        json_pred['categories']=dataset.class_names     
        json_pred['image']=filename.split(".")[0]
        json_pred['rois']=r['rois'].tolist()
        json_pred['labels']=r['class_ids'].tolist()
        json_pred['scores']=r['scores'].tolist()
        processed_list.append(filename)

        json_path=os.path.join(output_folder, filename.split(".")[0] + ".json")
        with open(os.path.join(json_path), "w+") as file:
          json.dump(json_pred, file)
      if save_masks:

        # create a single mask with category values
        complete_mask = np.zeros(img_arr.shape[:-1], dtype=np.uint8)
        for n in range(r['masks'].shape[2]):
          single_mask = r['masks'][:,:,n].astype(np.uint8)
          single_mask = single_mask*r['class_ids'][n]
          complete_mask=complete_mask+single_mask
        #save image
        mask_path=os.path.join(output_folder, "MASK-" + filename.split(".")[0] + ".png")
        imsave(mask_path, complete_mask)
        print(f"\n saved predicted masks for image {filename} as {mask_path}")
        processed_list.append(filename)
      
  proc_images_n=len(set(processed_list))
  print("processing finished !!")
  print(f"{proc_images_n} images elaborated")
  return set(processed_list)
  


In [ ]:
detect_flowers_fruits(input_images, output_folder, model, show_predictions=True, save_predictions=False )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img_dir = os.path.join(PROJECT_DIR,INPUT_DIR_NAME)
gt_json_path= os.path.join(PROJECT_DIR,INPUT_ANNOTATIONS_FILE)
 
 # create dataset
dataset = HerbariaDataset()
dataset.load_data(gt_json_path, img_dir)
dataset.prepare()
for image_id in dataset.image_ids[:2]:
  # load image and annotations from GT and detections
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
          modellib.load_image_gt(dataset, config,
                                  image_id)
  results = model.detect([image], verbose=0)
  r = results[0]
  # display gt
  visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, 
                                  dataset.class_names,figsize=(100,100), show_mask=False)
  # display pred
  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                  dataset.class_names,figsize=(100,100), show_mask=False)

# Model performance

In [ ]:
# Compute VOC-style Average Precision
def compute_batch_ap(image_ids):
    sp_eval = {}
    sp_eval['AP']=[]
    sp_eval['precisions']=[]
    sp_eval['recalls']=[]
    sp_eval['overlaps']=[]



    for image_id in image_ids:
        # Load image
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id)
        # Run object detection
        results = model.detect([image], verbose=0)
        # Compute AP
        r = results[0]
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                              r['rois'], r['class_ids'], r['scores'], r['masks'])
        sp_eval['AP'].append(AP)
        sp_eval['precisions'].append(precisions)
        sp_eval['recalls'].append(recalls)
        sp_eval['overlaps'].append(overlaps)
    return sp_eval

# # Pick a set of random images
# image_ids = np.random.choice(dataset.image_ids, 10)
# APs = compute_batch_ap(image_ids)
# print("mAP @ IoU=50: ", np.mean(APs))

In [ ]:
from utils import compute_matches

In [ ]:
from sklearn.metrics import f1_score
import seaborn as sns
sns.color_palette("Spectral", as_cmap=True)
def evaluate_model_performance(img_dir, gt_json_path, model, filter_score=0.5, viz=False):
  """ evaluates the model peroformance against a test set of images ad annotations
  Parameters:
  - model ??
  - detections folder with json files as retruned by <detect_flower_fruits> function
  - ground truth data (annotations) in COCO format

  OPTIONAL Parameters:
  -plot outputs
  -masks
  """


  # create dataset
  dataset = HerbariaDataset()
  dataset.load_data(gt_json_path, img_dir)
  dataset.prepare()
  
  # for det in os.listdir(detections_dir):
  #   if det.split(".") == "json":
  n_record=[]
  f1_boxes=[]
  recall_boxes=[]
  for image_id in dataset.image_ids:
  # load image and annotations from GT and detections
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config,
                                   image_id)
    results = model.detect([image], verbose=0)
    r = results[0]

    if filter_score:
        print(f"removing predictions below {filter_score}")
        sc_=r['scores']
        lim_=len(sc_[ sc_ > filter_score ])-1
        r['masks']=r['masks'][:,:,:lim_]
        r['rois']=r['rois'][:lim_]
        r['scores']= r['scores'][:lim_]
        r['class_ids'] = r['class_ids'][:lim_]
  # boxes classification
    max_len =min(len(gt_class_id), len(r['class_ids']))
    print(max_len)
    f1_=f1_score(gt_class_id[:max_len],r['class_ids'][:max_len])
    f1_boxes.append(f1_)
    print("\n f1 score : ", f1_)
  
  # Overlaps boxes and masks
  # sp_eval = compute_batch_ap(dataset.image_ids)
  # IOU masks
  
    iou_recall,_=utils.compute_recall(r['rois'], gt_bbox, 0)
    print("recall boxes:", iou_recall)
    recall_boxes.append(iou_recall)
  # break
  # compare areas
    # break


  print(f" mean f1 score of batch : {np.mean(f1_boxes)}")
  print(f"mean recall score of batch : {np.mean(recall_boxes)}")
  print(f1_boxes)

  if viz:
    pass
    
    
  eval_results={
      "f1_score": f1_boxes, 
      "recall_boxes": recall_boxes
                  
                  }
  return eval_results

img_dir = os.path.join(PROJECT_DIR,INPUT_DIR_NAME)
gt_json_path= os.path.join(PROJECT_DIR,INPUT_ANNOTATIONS_FILE)
perf=evaluate_model_performance(img_dir, gt_json_path, model, 0.5, False)


In [ ]:
import matplotlib
matplotlib.rc('axes',edgecolor="#FFFFFF")
matplotlib.rc('lines', color= "#FFFFFF")
# matplotlib.rc('xticks', color= "#FFFFFF")


sns.color_palette("Spectral", as_cmap=True)
fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(20,20))
fig.set_facecolor("#000000")
ax[0].hist(perf['f1_score'], bins=10, edgecolor='white', alpha=0.6)
# ax[0].set_facecolor("#000000")
ax[0].set_title("F1 score object classification")
ax[0].axvline(x=np.mean(perf['f1_score']), color="blue")
# ax[0].plot(perf['f1_score'], color='blue', alpha=0.6)

ax[1].hist(perf['recall_boxes'], bins=10, edgecolor='white', alpha=0.6)
ax[1].axvline(x=np.mean(perf['recall_boxes']), color="blue")

# ax[1].set_facecolor("#000000")
ax[0].xaxis.label.set_color("#FFFFFF")
ax[0].tick_params(axis='x', colors="#FFFFFF")
ax[1].tick_params(axis='x', colors="#FFFFFF")
ax[0].tick_params(axis='y', colors="#FFFFFF")
ax[1].tick_params(axis='y', colors="#FFFFFF")
ax[0].grid(axis='y',color="#000000", lw=2)
ax[1].grid(axis='y',color="#000000", lw=2)

plt.savefig('eval1.pdf')  


# Image analysis

In [ ]:
perf

In [ ]:
def analyse_batch_detections(image_folder, json_folder, mask_folder,):
  """" function to analys images AFTER detection. 
  Returns dictionary with data about prediction batch  
  Parameters:
  json_folder:  folder with the output of the model as produced by <detect_flower_fruits> function

  OPTIONAL parameters:
  mask_folder
  plot
  """
  # load image and data
  
  # flowers

  # image classification

  # flowers per image mean and stdev

  # fruits per image mean and standard dev

  pass



